## LoL Partidas Rankeadas

In [1]:
import numpy as np
import pandas as pd

# opções de display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# metodos de ajuda
from spells_info import get_spell, all_spells_names, get_spell_id
from champions_info import champions_info, champion_first_tag, champion_name, get_champion_id, all_champions_tags, is_s_tier, is_mechanical_intensive

### Lendo base da dados

In [2]:
dataset = pd.read_csv("./data/games.csv")

In [3]:
dataset.head()

,gameId,creationTime,gameDuration,seasonId,winner,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t1_towerKills,t1_inhibitorKills,t1_baronKills,t1_dragonKills,t1_riftHeraldKills,t1_ban1,t1_ban2,t1_ban3,t1_ban4,t1_ban5,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,t2_towerKills,t2_inhibitorKills,t2_baronKills,t2_dragonKills,t2_riftHeraldKills,t2_ban1,t2_ban2,t2_ban3,t2_ban4,t2_ban5
0,3326086514,1504279457970,1949,9,1,2,1,1,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,11,1,2,3,0,92,40,69,119,141,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,5,0,0,1,1,114,67,43,16,51
1,3229566029,1497848803862,1851,9,1,1,1,1,0,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,10,4,0,2,1,51,122,17,498,19,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,2,0,0,0,0,11,67,238,51,420
2,3327363504,1504360103310,1493,9,1,2,1,1,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,8,1,1,1,0,117,40,29,16,53,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,1,0,157,238,121,57,28
3,3326856598,1504348503996,1758,9,1,1,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,9,2,1,2,0,238,67,516,114,31,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,0,0,0,0,0,164,18,141,40,51
4,3330080762,1504554410899,2094,9,1,2,1,1,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,9,2,1,3,0,90,64,412,25,31,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,3,0,0,1,0,86,11,201,122,18


### Removendo colunas a posteriori e irrelevantes

- Removendo colunas que contem informações a posteriori(Numero de inibidores/torres/barões/dragões destruidas)
- Ou que não devem fazer diferença durante o jogo(bans/id do jogo/data de criação do jogo).

In [4]:
# Colunas que contem informação a posteriori do jogo
apost_columns = ['gameDuration', 'firstInhibitor', 'firstBaron',
        't1_towerKills', 't1_inhibitorKills', 't1_baronKills',
        't1_dragonKills', 't1_riftHeraldKills', 't2_towerKills',
         't2_inhibitorKills', 't2_baronKills', 't2_dragonKills', 
         't2_riftHeraldKills']

# Champs banidos não influenciam no resultado dos jogos
bans_columns = ['t1_ban1', 't1_ban2', 't1_ban3',
                't1_ban4', 't1_ban5', 't2_ban1',
                't2_ban2', 't2_ban3','t2_ban4',
                't2_ban5']

# Colunas que não agregam informação ao problema
remove_columns = ['gameId', 'creationTime', 'seasonId'] # Todos os jogos são da season 9

# As demais colunas contem informação de até 15minutos dentro do jogo

In [5]:
dataset.drop(columns=apost_columns, inplace=True)
dataset.drop(columns=bans_columns, inplace=True)
dataset.drop(columns=remove_columns, inplace=True)

In [6]:
dataset.head()

,winner,firstBlood,firstTower,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2
0,1,2,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3
1,1,1,1,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7
2,1,2,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4
3,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7
4,1,2,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7


### Adicionando features com bases nos campeões/spells escolhidos

Iremos adicionar as colunas que conta o número de spells por time/tipo de campeões

In [7]:
def aggregate_spells(name, dataframe):
    names = {}
    for n in all_spells_names():
        if n == name:
            names[get_spell_id(n)] = 1
        else:
            names[get_spell_id(n)] = 0
    
    df = dataframe.replace(names)
    new_column = df.agg("sum", axis="columns")
    
    return new_column

In [8]:
team_one_spells = ["t1_champ1_sum1", "t1_champ2_sum1", "t1_champ3_sum1", 
                   "t1_champ4_sum1", "t1_champ5_sum1", "t1_champ1_sum2", 
                   "t1_champ2_sum2", "t1_champ3_sum2", "t1_champ4_sum2", 
                   "t1_champ5_sum2"]

team_two_spells = ["t2_champ1_sum1", "t2_champ2_sum1", "t2_champ3_sum1", 
                   "t2_champ4_sum1", "t2_champ5_sum1", "t2_champ1_sum2",
                   "t2_champ2_sum2", "t2_champ3_sum2", "t2_champ4_sum2", 
                   "t2_champ5_sum2"]

In [9]:
# t1
for spell in all_spells_names():
    dataset["count_" + spell + "_t1"] = aggregate_spells(spell, dataset[team_one_spells])
# t2
for spell in all_spells_names():
    dataset["count_" + spell + "_t2"] = aggregate_spells(spell, dataset[team_two_spells])

In [10]:
dataset.head()

,winner,firstBlood,firstTower,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,count_Teleport_t1,count_Cleanse_t1,count_Ghost_t1,count_Barrier_t1,count_Exhaust_t1,count_Ignite_t1,count_Smite_t1,count_Heal_t1,count_Flash_t1,count_Teleport_t2,count_Cleanse_t2,count_Ghost_t2,count_Barrier_t2,count_Exhaust_t2,count_Ignite_t2,count_Smite_t2,count_Heal_t2,count_Flash_t2
0,1,2,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,1,0,1,0,1,1,1,1,4,0,0,1,0,1,1,1,1,5
1,1,1,1,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,1,0,0,0,1,1,1,1,5,1,0,0,0,0,2,1,1,5
2,1,2,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,0,1,0,1,1,5,1,0,0,0,0,1,1,2,5
3,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,1,1,0,0,0,1,1,1,5,0,0,0,0,1,2,1,1,5
4,1,2,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,1,0,0,0,1,0,1,2,5,2,0,0,0,1,0,1,1,5


In [11]:
def aggregate_tags(name, dataframe):
    names = {}
    for champion_id in champions_info:
        if name == champion_first_tag(champion_id):
            names[champion_id] = 1
        else:
            names[champion_id] = 0
    
    df = dataframe.replace(names)
    new_column = df.agg("sum", axis="columns")
    
    return new_column

In [12]:
team_one_champions = ["t1_champ1id", "t1_champ2id", "t1_champ3id", 
                      "t1_champ4id", "t1_champ5id"]

team_two_champions = ["t2_champ1id", "t2_champ2id", "t2_champ3id", 
                      "t2_champ4id", "t2_champ5id"]

In [13]:
# t1
for tag in all_champions_tags():
    dataset["count_" + tag + "_t1"] = aggregate_tags(tag, dataset[team_one_champions])
# t2
for tag in all_champions_tags():
    dataset["count_" + tag + "_t2"] = aggregate_tags(tag, dataset[team_two_champions])

In [14]:
dataset.head()

,winner,firstBlood,firstTower,firstDragon,firstRiftHerald,t1_champ1id,t1_champ1_sum1,t1_champ1_sum2,t1_champ2id,t1_champ2_sum1,t1_champ2_sum2,t1_champ3id,t1_champ3_sum1,t1_champ3_sum2,t1_champ4id,t1_champ4_sum1,t1_champ4_sum2,t1_champ5id,t1_champ5_sum1,t1_champ5_sum2,t2_champ1id,t2_champ1_sum1,t2_champ1_sum2,t2_champ2id,t2_champ2_sum1,t2_champ2_sum2,t2_champ3id,t2_champ3_sum1,t2_champ3_sum2,t2_champ4id,t2_champ4_sum1,t2_champ4_sum2,t2_champ5id,t2_champ5_sum1,t2_champ5_sum2,count_Teleport_t1,count_Cleanse_t1,count_Ghost_t1,count_Barrier_t1,count_Exhaust_t1,count_Ignite_t1,count_Smite_t1,count_Heal_t1,count_Flash_t1,count_Teleport_t2,count_Cleanse_t2,count_Ghost_t2,count_Barrier_t2,count_Exhaust_t2,count_Ignite_t2,count_Smite_t2,count_Heal_t2,count_Flash_t2,count_Mage_t1,count_Fighter_t1,count_Tank_t1,count_Assassin_t1,count_Marksman_t1,count_Support_t1,count_Mage_t2,count_Fighter_t2,count_Tank_t2,count_Assassin_t2,count_Marksman_t2,count_Support_t2
0,1,2,1,1,2,8,12,4,432,3,4,96,4,7,11,11,6,112,4,14,104,11,4,498,4,7,122,6,4,238,14,4,412,4,3,1,0,1,0,1,1,1,1,4,0,0,1,0,1,1,1,1,5,2,0,0,1,1,1,0,1,0,1,2,1
1,1,1,1,1,1,119,7,4,39,12,4,76,4,3,10,4,14,35,4,11,54,4,12,25,4,14,120,11,4,157,4,14,92,4,7,1,0,0,0,1,1,1,1,5,1,0,0,0,0,2,1,1,5,0,2,0,2,1,0,1,3,1,0,0,0
2,1,2,1,2,0,18,4,7,141,11,4,267,3,4,68,4,12,38,12,4,69,4,7,412,14,4,126,4,12,24,4,11,22,7,4,2,0,0,0,1,0,1,1,5,1,0,0,0,0,1,1,2,5,0,2,0,1,1,1,1,2,0,0,1,1
3,1,1,1,1,0,57,4,12,63,4,14,29,4,7,61,4,1,36,11,4,90,14,4,19,11,4,412,4,3,92,4,14,22,4,7,1,1,0,0,0,1,1,1,5,0,0,0,0,1,2,1,1,5,2,1,1,0,1,0,1,2,0,0,1,1
4,1,2,1,1,0,19,4,12,29,11,4,40,4,3,119,4,7,134,7,4,37,3,4,59,4,12,141,11,4,38,4,12,51,4,7,1,0,0,0,1,0,1,2,5,2,0,0,0,1,0,1,1,5,1,1,0,0,2,1,0,1,1,1,1,1


### Removendo as colunas de campeões e spells

In [15]:
# spells
dataset.drop(columns=team_one_spells, inplace=True)
dataset.drop(columns=team_two_spells, inplace=True)
# campeões
dataset.drop(columns=team_one_champions, inplace=True)
dataset.drop(columns=team_two_champions, inplace=True)

In [16]:
dataset.head()

,winner,firstBlood,firstTower,firstDragon,firstRiftHerald,count_Teleport_t1,count_Cleanse_t1,count_Ghost_t1,count_Barrier_t1,count_Exhaust_t1,count_Ignite_t1,count_Smite_t1,count_Heal_t1,count_Flash_t1,count_Teleport_t2,count_Cleanse_t2,count_Ghost_t2,count_Barrier_t2,count_Exhaust_t2,count_Ignite_t2,count_Smite_t2,count_Heal_t2,count_Flash_t2,count_Mage_t1,count_Fighter_t1,count_Tank_t1,count_Assassin_t1,count_Marksman_t1,count_Support_t1,count_Mage_t2,count_Fighter_t2,count_Tank_t2,count_Assassin_t2,count_Marksman_t2,count_Support_t2
0,1,2,1,1,2,1,0,1,0,1,1,1,1,4,0,0,1,0,1,1,1,1,5,2,0,0,1,1,1,0,1,0,1,2,1
1,1,1,1,1,1,1,0,0,0,1,1,1,1,5,1,0,0,0,0,2,1,1,5,0,2,0,2,1,0,1,3,1,0,0,0
2,1,2,1,2,0,2,0,0,0,1,0,1,1,5,1,0,0,0,0,1,1,2,5,0,2,0,1,1,1,1,2,0,0,1,1
3,1,1,1,1,0,1,1,0,0,0,1,1,1,5,0,0,0,0,1,2,1,1,5,2,1,1,0,1,0,1,2,0,0,1,1
4,1,2,1,1,0,1,0,0,0,1,0,1,2,5,2,0,0,0,1,0,1,1,5,1,1,0,0,2,1,0,1,1,1,1,1


### Salvando base de dados preprocessada

In [20]:
dataset.to_csv("./data/base_jogos.csv", index=None)